In [3]:
import torch
import random

首先要实现人工数据集，写一个生成数据集的函数：

In [4]:
def synthetic_data(w, b, num):
    '''
    params:
        w: 权重
        b: 偏置
        num: 数据量
    return:
        X: 输入
        y: 输出
    '''
    X = torch.normal(0, 1, (num, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape(num, -1)
    

测试：

In [5]:
w = torch.tensor([2, 1.0])
b = float(3.0)
X, y = synthetic_data(w, b, 10)
X.shape, y.shape, len(y)

(torch.Size([10, 2]), torch.Size([10, 1]), 10)

然后构造损失函数：

In [6]:
def loss_fn(y, y_hat):
    '''
    params:
        y: 标签
        y_hat: 拟合值
    return:
        loss: 平方损失
    '''
    return torch.matmul((y-y_hat).T, y-y_hat) / len(y)

构造线性回归（拟合）模型：

In [7]:
def linear_model(w, x, b):
    '''
    params:
        w: 权重
        x: 输入
        b: 偏置
    return:
        y: 输出
    '''
    return torch.matmul(x, w) + b

实现梯度下降算法：

In [8]:
def sgd(params, lr):
    '''
    params:
        params: 拟合的参数
        lr: 学习率
    '''
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad
            param.grad.zero_()

数据随机读取：

In [13]:
def data_iter(batch_size,features,labels):
    '''
    params:
        batch_size: 批量
        features: 输入
        labels: 标准输出
    return:
        batch_features: 批量输入
        batch_labels: 批量输出
    '''
    num_examples = len(features)  # 样本个数
    indices = list(range(num_examples)) # 样本索引
    # 这些样本是随即读取的，没有特定的顺序
    random.shuffle(indices) # 把索引随即打乱
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num_examples)]) # 当i+batch_size超出时，取num_examples    
        yield features[batch_indices], labels[batch_indices] # 获得随即顺序的特征，及对应的标签

开始训练

In [14]:
epoch_num = 5
batch_size = 10
lr = 0.2
w_real = torch.tensor([2.0, 1.0])
b_real = 3.0
num = 1000
features, labels = synthetic_data(w_real, b_real, num)

w = torch.normal(0, 1, (2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
for epoch in range(epoch_num):
    for X,y in data_iter(batch_size,features,labels):
        l = loss_fn(linear_model(w, X, b),y) # x和y的小批量损失
        # 因为l是形状是(batch_size,1)，而不是一个标量。l中所有元素被加到一起
        # 并以此计算关于[w,b]的梯度
        l.backward()
        sgd([w,b],lr) #使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss_fn(linear_model(w, features, b), labels)
        print(f'epoch: {epoch+1},loss: {float(train_l):f}...') 

epoch: 1,loss: 0.000118...
epoch: 2,loss: 0.000119...
epoch: 3,loss: 0.000106...
epoch: 4,loss: 0.000106...
epoch: 5,loss: 0.000103...


完毕

补充：python的List是不能进行 List[[1,3,5]] 的操作的，但是似乎 tensor 张量可以，进行测试：

In [15]:
a = [0,1,2,3,4,5]
a[[1,3,5]]

TypeError: list indices must be integers or slices, not list

In [18]:
b = torch.tensor([0,1,2,3,4,5])
b[[1,3,5]]

tensor([1, 3, 5])

总结：牛的。